# Job Hop Net Data Preparation
## A. Mazzetto
### December 2022

In [ ]:
from datetime import date, timedelta
from dateparser import parse
import itertools
import re
import time
import pickle

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
import hashlib
from gensim.parsing.preprocessing import remove_stopwords
from deep_translator import GoogleTranslator

# Data Preparation

Please skip to the Data Analysis section if not interested in the data preparation. This might take some time due to the Google Translation and Google NGram query. Note that this is currently based on a tiny part of the dataset, jsut for exemplification.

In [ ]:
# Load the data
rawdata0 = pd.read_csv('..\data\dataset-tiny-sample.csv', index_col= 0)

# Names are often present only for the first work esperience
rawdata0['Name'] = rawdata0['Name'].fillna(method='ffill')

# Drop rows with NA
rawdata0 = rawdata0.dropna()
rawdata0.head()

In [ ]:
%%script false --no-raise-error
# These are commented as they were useful only on the original, non-anonymous dataset

# Drop columnd Serial No. as not reliable
rawdata0 = rawdata0.drop(columns= ['1'])
# Anonymize the dataset
rawdata0['Name'] = rawdata0['Name'].apply(lambda x: hashlib.sha1(x.encode("utf-8")).hexdigest())

In [ ]:
rawdata0.shape

# Date Parsing

In [ ]:
# Parse dates: substitute string 'Present' with current date
rawdata1 = rawdata0.copy()
rawdata1['To'][rawdata1['To'].str.lower().str.find('present')>-1] = date.today().strftime("%Y-%m") # YM
rawdata1.head()

In [ ]:
# Parse dates: transform two-digit numbers into year if there is a string for the month
current_year_int = int(str(date.today().year)[-2:])
def expand_year(match):
    token = match.group(2)
    token_new = ''
    if int(token) <= current_year_int:
        if len(match.group(2)) == 1:
            token_new = '200' + token
        else:
            token_new = '20' + token
    else:
        token_new = '19' + token
    return match.group(1) + token_new + match.group(3)
rawdata1[['From', 'To']] = rawdata1[['From', 'To']].applymap(
    lambda x: re.compile('(.*[a-zA-Z]+.*[^\d]|^)(\d{1,2})([^\d].*[a-zA-Z]+.*|$)').sub(expand_year, str(x)))
rawdata1.head()

In [ ]:
# Parse date: use dateparser
config_dateparser = {'PREFER_DAY_OF_MONTH': 'first'}
rawdata1[['From', 'To']] = rawdata1[['From', 'To']].applymap(
    lambda d: parse(str(d), settings= config_dateparser))
rawdata1.head()

In [ ]:
# Display the dates that could not be parsed correctly
print('Observations with dates that could not be parsed correctly')
display(rawdata0[rawdata1[['From', 'To']].isnull().values.any(axis= 1)])
# Remove the datest that could not be parsed correctly
rawdata1.drop(rawdata1[rawdata1[['From', 'To']].isnull().values.any(axis= 1)].index, inplace= True)

In [ ]:
# recalculate Experience column based on parsing
rawdata2 = rawdata1.copy()
rawdata2['Experience'] = rawdata2['To'] - rawdata2['From']

# Check for negative periods (usually are typos) by looking at the original dataset
print('Observaions with wrong or missing chronological order')
display(rawdata0.loc[rawdata2[rawdata2['Experience'] < timedelta(0)].index])

# Remove the observations with wrong cronological order
rawdata2.drop(rawdata2[rawdata2['Experience'] < timedelta(0)].index, inplace= True)
rawdata2.head()

In [ ]:
rawdata2.shape

In [ ]:
print(f'There are {rawdata2.Name.unique().shape[0]} different profiles')
print(f'There are {rawdata2.Company.unique().shape[0]} different firms')
print(f'There are {rawdata2.Role.unique().shape[0]} different roles')
print(f'There are {rawdata2.shape[0] - rawdata2.Name.unique().shape[0]} job hops')

# Job and Company parsing

In [ ]:
rawdata3 = rawdata2.copy()

# Translate jobs if necessary
googlet = GoogleTranslator(source='auto', target='en')
rawdata3[['Role', 'Company']] = rawdata1[['Role', 'Company']].applymap(
    lambda x: googlet.translate(x))
rawdata3.head()

In [ ]:
# Check which jobs got translated
italian_jobs = rawdata2[(rawdata3[['Role', 'Company']] != rawdata2[['Role', 'Company']]).any(axis= 1)][['Role', 'Company']]
italian_jobs = italian_jobs.rename(columns= {'Role': 'Italian Role', 'Company': 'Italian Company'})
english_jobs = rawdata3[(rawdata3[['Role', 'Company']] != rawdata2[['Role', 'Company']]).any(axis= 1)][['Role', 'Company']]
pd.concat([italian_jobs, english_jobs], axis= 1)

In [ ]:
# Make all lower case
print(f'There are NAs: {str(np.where(rawdata3.isna()))}')
rawdata3 = rawdata3.dropna()
# Remove punctuation
rawdata3[['Role', 'Company']] = rawdata3[['Role', 'Company']].applymap(lambda x: re.sub(r'[^\w\s]', '', x))
# Make lower case
rawdata3[['Role', 'Company']] = rawdata3[['Role', 'Company']].applymap(str.lower)
# Remove stop-words
rawdata3[['Role', 'Company']] = rawdata3[['Role', 'Company']].applymap(lambda x: remove_stopwords(x))

### In the next session we swap Company and Role columns where necessary

In [ ]:
# List of jobs from https://www.careerbuilder.com/browse
jobs_list= []
jobs_from_website = pd.read_csv('..\data\jobs-from-website.csv', header= None)
# Function to extract jobs from strings
for i in jobs_from_website.values:
    temporary = re.split(r'([a-z])([A-Z])',i[0])
    if len(temporary) > 3:
        jobs_list += [temporary[0] + temporary[1]]
        jobs_list += [temporary[-2] + temporary[-1]]
    else:
        jobs_list += temporary
    if len(temporary) > 4:
        for j in range(3,len(temporary)-1,3):
            jobs_list += [temporary[j-1] + temporary[j] + temporary[j+1]]
# Take only the last word from the job title
job_titles = [i.split()[-1] for i in jobs_list]
# Some other common job-related words
job_titles.extend(['manager', 'director', 'chief', 'president', 'head', 'partner', 'leader', \
    'founder', 'senior', 'member', 'chairman', 'ceo', 'deputy', 'lead'\
    'responsible', 'collaborator', 'crew', 'recruiter', 'planner', 'developer', 'author', \
    'consultant', 'specialist', 'generalist', 'analyst', 'trainer', 'associate', 'officer', 'advisor'\
    'dealer', 'teller', 'coach', 'talent', 'engineer', 'lecturer', 'fellow', 'tutor', 'secretary', \
    'hostess', 'junior', 'assistant', 'trainee', 'student', 'intern', 'graduate', 'scholar'\
    'merchandiser', 'draftsman', 'controller', 'executive', 'designer', 'technician', 'buyer', \
    'researcher', 'worker', 'architect', 'department', 'responsible', 'internship', 'engineer', \
    'scientist', 'operator', 'machinist', 'mechanic', 'installer', 'auditor', 'counsel', 'assessor'])
job_list_unique = list(set([i.lower() for i in job_titles]))

In [ ]:
# Swap Company with Role if necessary
rawdata4 = rawdata3.copy()
swap_company_role = rawdata3['Company'].apply(lambda x: any([i in job_list_unique for i in x.split()])) & \
    rawdata3['Role'].apply(lambda x: any([i not in job_list_unique for i in x.split()]))
rawdata4.loc[swap_company_role, ['Role', 'Company']] = rawdata4.loc[swap_company_role, ['Company', 'Role']].values
# Check swaps
pd.concat([rawdata3.loc[swap_company_role, ['Company', 'Role']], 
           rawdata4.loc[swap_company_role, ['Company', 'Role']]], axis= 1)

In [ ]:
# Remove duplicates based on 'Name', 'Company', 'From' and 'To'
rawdata4 = rawdata4[~rawdata4[['Name','Company','From','To']].duplicated(keep= 'first')]

## Normalize Companies

In [ ]:
print(f'There are {rawdata4.Name.unique().shape[0]} different profiles')
print(f'There are {rawdata4.Company.unique().shape[0]} different firms')
print(f'There are {rawdata4.Role.unique().shape[0]} different roles')
print(f'There are {rawdata4.shape[0] - rawdata4.Name.unique().shape[0]} job hops')

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

### Count vectorization and clustering

In [ ]:
cv = CountVectorizer()
cvfit = cv.fit_transform(rawdata4['Company'])

In [ ]:
with np.printoptions(threshold=np.inf):
    for i in list(zip(sorted(cv.vocabulary_),cvfit.sum(axis= 0).tolist()[0])):
        if i[1] >= 100 and i[1] < np.inf:
            print(i)

In [ ]:
n_clusters = min(200, rawdata4.shape[0] - 1)
kmeans_cv = KMeans(n_clusters=n_clusters, random_state= 892536, n_init= 10)
kmeans_cv.fit(cvfit)
clusters_cv = kmeans_cv.labels_

In [ ]:
# By manually inspecting the clusters it is possible to see that many do not make sense
with np.printoptions(threshold=np.inf):
    print(rawdata4['Company'][clusters_cv==0].unique())

### TF-IDF vectorization and clustering

In [ ]:
vectorizer = TfidfVectorizer( sublinear_tf= True, stop_words= 'english')

In [ ]:
X= vectorizer.fit_transform(rawdata4['Company'])

In [ ]:
kmeans = KMeans(n_clusters= n_clusters, random_state= 892536, n_init= 10)
kmeans.fit(X)
clusters = kmeans.labels_

In [ ]:
# By manually inspecting the clusters these make much more sense than before
with np.printoptions(threshold=np.inf):
    print(rawdata4['Company'][clusters==0].unique())

### Doc2Vec vectorization

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
company_documents = rawdata4['Company'].apply(str.split).tolist()

In [ ]:
company_documents_tagged = [TaggedDocument(d, [i]) for i, d in enumerate(company_documents)]

In [ ]:
vector_size = 20
model_d2v = Doc2Vec(company_documents_tagged, min_count= 2, vector_size= vector_size, window=1, workers= 4)

In [ ]:
document_vectors = np.ndarray((0,vector_size))
for i in range(len(model_d2v.dv)):
    document_vectors = np.vstack((document_vectors,model_d2v.dv.get_vector(i).reshape(1,-1)))

In [ ]:
kmeans_d2v = KMeans(n_clusters= n_clusters, random_state= 892536, n_init= 100)
kmeans_d2v.fit(document_vectors)
clusters_d2v = kmeans_d2v.labels_

In [ ]:
# By manually inspecting the clusters these do not make much sense
with np.printoptions(threshold=np.inf):
    print(rawdata4['Company'][clusters_d2v==0].unique())

### Google ngram approach

In [ ]:
# Google NGram approach
import requests
import urllib
  
def google_ngram_query(query, start_year=2018, 
             end_year=2019, corpus= 26,
             smoothing=2):
  
    # converting a regular string to 
    # the standard URL format
    # eg: "geeks for,geeks" will
    # convert to "geeks%20for%2Cgeeks"
    query = urllib.parse.quote(query)
  
    # creating the URL
    url = 'https://books.google.com/ngrams/json?content=' + query + \
    '&year_start=' + str(start_year) + '&year_end=' + \
    str(end_year) + '&corpus=' + str(corpus) + '&smoothing=' + \
    str(smoothing) + ''
  
    # requesting data from the above url
    response = requests.get(url)
    it = 0
    while it < 10 and response.status_code != 200:
        response = requests.get(url)
        it += 1
  
    # extracting the json data from the response we got
    return_data = {}
    if response.status_code == 200:
        output = response.json()
        if len(output) == 0:
            # if no data returned from Google
            print('No data available for this Ngram.')
        else:
            # if data returned from Google
            for num in range(len(output)):
                return_data[output[num]['ngram']] = output[num]['timeseries'][-1]
    else:
        print('Stale request!')
  
    return return_data

In [ ]:
# Test the HTML
t0 = time.time()
print(google_ngram_query('house'))
print(time.time() - t0)

In [ ]:
# Algorithm to keep the most unsual word
def keep_unusual_word(x):
    if not isinstance(x,str):
        return(x)
    x_out = x.strip().split()
    if len(x_out) > 1:
        ngram_dict = google_ngram_query(','.join(x_out))
        if len(ngram_dict) > 0:
            x_out = min(ngram_dict, key= ngram_dict.get)
        else:
            x_out = ' '.join(x_out)
    elif len(x_out) == 1:
        x_out = x_out[0]
    else:
        x_out = np.nan
    return(x_out)

In [ ]:
# Run the code and keep only most unusual word: not that we get some stale requests
rawdata4['Company'] = rawdata4['Company'].apply(keep_unusual_word)

In [ ]:
# Check the results: stale requests meant that some jobs are not parsed correctly
rawdata4['Company'].value_counts()[0:10]

In [ ]:
# By how much the number got reduced
len(rawdata3['Company'].unique()), len(rawdata4['Company'].unique())

### Manual Company Normalization

In [ ]:
# Undo changes done before
rawdata4 = rawdata3.copy()

In [ ]:
# Manual company normalization
firms = pd.read_csv(r'..\data\firms-tiny-sample.csv', keep_default_na= False)
firms_names = dict(zip(firms['Company'],firms['Alias']))
# Anonymize Italian Motor Valley companies
firms_new_names = [row['Alias'] if row['Alias']!='' else row['Company'] for irow, row in firms.iterrows()]
firms_number = dict(zip(firms_new_names,firms['Number of Employees']))
firms_sector = dict(zip(firms_new_names,firms['Type']))
firms_names

In [ ]:
# Function to look for nicknames and replace
def company_normalization(x):
    out = None
    for i in firms_names.keys():
        if re.search(i,x):
            if firms_names[i]:
                out = firms_names[i]
            else:
                out = i
    return(out)

In [ ]:
rawdata5 = rawdata4.copy()
rawdata5['Company'] = rawdata5['Company'].apply(lambda x: company_normalization(x))
rawdata5.dropna(inplace= True)
rawdata5.head()

In [ ]:
print(f'There are {rawdata5.Name.unique().shape[0]} different profiles')
print(f'There are {rawdata5.Company.unique().shape[0]} different firms')
print(f'There are {rawdata5.Role.unique().shape[0]} different roles')
print(f'There are {rawdata5.shape[0] - rawdata4.Name.unique().shape[0]} job hops')

## Role Normalization

This section was tested but not used nor reported

In [ ]:
%%script false --no-raise-error

role_documents = rawdata4['Role'].apply(str.split).tolist()
role_documents_tagged = [TaggedDocument(d, [i]) for i, d in enumerate(role_documents)]
role_d2v = Doc2Vec(role_documents_tagged, min_count= 2, vector_size= vector_size, window=1, workers= 4)

In [ ]:
%%script false --no-raise-error

role_vectors = np.ndarray((0,vector_size))
for i in range(len(model_d2v.dv)):
    role_vectors = np.vstack((role_vectors,role_d2v.dv.get_vector(i).reshape(1,-1)))

In [ ]:
%%script false --no-raise-error

kmeans_role_d2v = KMeans(n_clusters= 200, random_state= 892536, n_init= 100)
kmeans_role_d2v.fit(role_vectors)
clusters_role_d2v = kmeans_role_d2v.labels_

In [ ]:
%%script false --no-raise-error

with np.printoptions(threshold=np.inf):
    print(rawdata4['Role'][clusters_role_d2v==0].unique())

# Prepare Job Network

In [ ]:
# Remove people with only one experience
rawdata5 = rawdata5[rawdata5['Name'].duplicated(keep= False)]

In [ ]:
print(f'There are {rawdata5.Name.unique().shape[0]} different profiles')
print(f'There are {rawdata5.Company.unique().shape[0]} different firms')
print(f'There are {rawdata5.Role.unique().shape[0]} different roles')
print(f'There are {rawdata5.shape[0] - rawdata5.Name.unique().shape[0]} job hops')

In [ ]:
# Algorithm to extract job hops
job_hops = []
job_hops_ext = []
role_hops = []
role_hops_ext = []
for name in rawdata5['Name'].unique():
    df = rawdata5[rawdata5['Name']==name].sort_values(by= 'From')
    row_queue = []
    for i, (irow, row) in enumerate(df.iterrows()):
        if i != 0:
            for j in range(len(row_queue)):
                if row['From'] >= row_queue[j]['To']:
                    job_hops.append((row_queue[j]['Company'], row['Company']))
                    role_hops.append((row_queue[j]['Role'], row['Role']))
                    if row_queue[j]['Company'] != row['Company']:
                        job_hops_ext.append((row_queue[j]['Company'], row['Company']))
                    if row_queue[j]['Role'] != row['Role']:
                        role_hops_ext.append((row_queue[j]['Role'], row['Role']))
                    row_queue.pop(j)
                    break      
        row_queue.append(row)

In [ ]:
len(job_hops), len(job_hops_ext)

In [ ]:
len(role_hops), len(role_hops_ext)

In [ ]:
# Unique job hops
job_hop_ext_unique = list(set(job_hops_ext))
job_hop_ext_unique_wt = [job_hops_ext.count(i) for i in job_hop_ext_unique]
job_hop_ext_unique_weighted = [i1 + (dict(weight = i2/firms_number[i1[0]], weight_tgt = i2/firms_number[i1[1]]),) \
                               for i1, i2 in zip(job_hop_ext_unique, job_hop_ext_unique_wt)]
print(f'There are {len(job_hops_ext)} job hops of which {len(job_hop_ext_unique)} are unique')

`job_hop_ext_unique`, `job_hop_ext_unique_wt` and `job_hop_ext_unique_weighted` define the network

In [ ]:
with open('../data/job-hop-net.dat','w') as file:
    for job_link, weight in zip(job_hop_ext_unique, job_hop_ext_unique_wt):
        file.write('\t'.join((
            job_link[0],
            firms_sector[job_link[0]],
            job_link[1],
            firms_sector[job_link[1]],
            str(weight),
            str(weight/firms_number[job_link[0]]),
            str(weight/firms_number[job_link[1]]))
        ) + '\n')
    file.close()

### End of file